In [266]:
import tensorflow as tf
import os

from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras import regularizers
from keras.layers import LeakyReLU
keras = tf.keras  

In [267]:
gpus = tf.config.experimental.list_physical_devices('CPU')


In [268]:
gpus

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [269]:
gpus = tf.config.experimental.list_physical_devices ('GPU') 
for gpu in gpus:
    tf.config.experimental.set_memory_growth (gpu, True)

In [270]:
import cv2
import imghdr
from matplotlib import pyplot as plt

In [271]:
data_dir=r"/kaggle/input/flag-pictures-dataset/Flags"

In [272]:
image_exts=["jpeg","jpg","bmp","png"]

In [359]:
num_classes=len(os.listdir(r"/kaggle/input/flag-pictures-dataset/Flags"))

In [360]:
for image_class in os.listdir(data_dir):
    for image in os.listdir (os.path.join(data_dir, image_class)): 
        image_path = os.path.join(data_dir, image_class, image) 
        try:
            img= cv2.imread(image_path)
            tip=imghdr.what (image_path)
            if tip not in image_exts:
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e:
            print('Issue with image {}'.format(image_path))

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: cHRM: inconsistent chromaticities
libpng warning: cHRM: inconsistent chromaticities
libpng warning: iCCP: known incorrect sRGB profile


In [275]:
import numpy as np

In [276]:
datagen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,         # Randomly zoom images by 20%
    width_shift_range=0.1,  # Randomly shift images horizontally by 10%
    height_shift_range=0.1, # Randomly shift images vertically by 10%
    rotation_range=5,      # Randomly rotate images by up to 20 degrees
    brightness_range=(0.7, 1.3),  # Randomly adjust brightness between 0.8 and 1.2
    shear_range=0.2,        # Randomly apply shearing transformations
    horizontal_flip=False,   # Randomly flip images horizontally
    vertical_flip=False      # Randomly flip images vertically
)

In [361]:
data = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    label_mode='categorical',  # This indicates multiclass classification
    seed=42,  # Set the seed for reproducibility
    image_size=(256, 256),
    batch_size=32
)

Found 3214 files belonging to 18 classes.


In [278]:
data=data.map(lambda x,y:(x/255,y))

In [279]:
scaled_iterator=data.as_numpy_iterator()

In [280]:
batch=scaled_iterator.next()

In [281]:
train_size = int(len(data) * 0.8)
val_size = int(len(data) * 0.1)
test_size = int(len(data) * 0.1)

In [325]:
train=data.take (train_size)
val = data.skip(train_size).take(val_size) 
test=data.skip (train_size+val_size).take(test_size)

In [283]:
# def preprocess_image(image_path):
#     img = cv2.imread(image_path)
#     tip = imghdr.what(image_path)
#     if tip not in image_exts:
#         print('Image not in ext list {}'.format(image_path))
#         os.remove(image_path)
#         return None
#     # Convert to 8-bit unsigned integers
#     img = cv2.convertScaleAbs(img)
#     return img

In [362]:
import tensorflow as tf

# Define the data augmentation function
def augment_image(image, label):
    # Randomly adjust brightness, contrast, saturation, and hue
    image = tf.image.random_brightness(image, max_delta=0.02)
    image = tf.image.random_contrast(image, lower=0.99, upper=1.01)
    image = tf.image.random_saturation(image, lower=0.99, upper=1.01)
    image = tf.image.random_hue(image, max_delta=0.01)
    # Add more augmentation techniques as per your requirement
    # For example, you can use tf.image.random_flip_left_right for random horizontal flipping, etc.
    
    return image, label

# Define the preprocess function without resizing to use it later for validation and test sets
def preprocess_image(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.uint8)  # Convert to 8-bit unsigned int
    image = tf.image.resize(image, (256, 256))
    image =image/255.0# Resize to model input size
    return image, label

# Apply duplication and augmentation to the training data
# def augment_and_duplicate(image, label, num_duplicates=1):
#     # List to store augmented images and labels
#     augmented_images = []
#     augmented_labels = []
    
#     # Add original image and label to the lists
#     augmented_images.append(image)
#     augmented_labels.append(label)
    
#     # Apply augmentation and add duplicates to the lists
#     for _ in range(num_duplicates):
#         augmented_image, augmented_label = augment_image(image, label)
#         augmented_images.append(augmented_image)
#         augmented_labels.append(augmented_label)
    
#     # Stack the lists to create tensors
#     augmented_images = tf.stack(augmented_images)
#     augmented_labels = tf.stack(augmented_labels)
    
#     return augmented_images, augmented_labels

# # Duplicate and augment the training data
# num_duplicates = 0 # You can adjust the number of duplicates as per your requirement
# train = train.flat_map(lambda image, label: tf.data.Dataset.from_tensor_slices(augment_and_duplicate(image, label, num_duplicates)))
train = train.map(preprocess_image)
# Apply preprocessing to validation and test sets
val = val.map(preprocess_image)
test = test.map(preprocess_image)


In [ ]:
train.as_numpy_iterator().next()[0]

In [286]:
model=Sequential()

In [287]:
model = Sequential()
model.add(Conv2D(32, (7, 7), 1, activation=keras.layers.LeakyReLU(alpha=0.2), input_shape=(256, 256, 3)))
model.add(MaxPooling2D())


# Second convolution layer with increased filters, Dropout, and Regularization
model.add(Conv2D(64, (5, 5), strides=1, activation=keras.layers.LeakyReLU(alpha=0.1), kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.25))
model.add(MaxPooling2D())

# Third convolution layer with increased filters and BatchNormalization
model.add(Conv2D(32, (3, 3), strides=1, activation=keras.layers.LeakyReLU(alpha=0.05), kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPooling2D())

# Flatten the output to feed into the dense layers
model.add(Flatten())

# Dense layers with Dropout and Regularization
model.add(Dense(256, activation=keras.layers.LeakyReLU(alpha=0.02), kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.5))

# Output layer for multiclass classification
model.add(Dense(num_classes, activation='softmax'))

early_stopping=tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=1,
    verbose=1,
    mode='min',
    baseline=None,
    restore_best_weights=True,
    start_from_epoch=0
)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'],)



In [288]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 250, 250, 32)      4736      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 125, 125, 32)     0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 121, 121, 64)      51264     
                                                                 
 dropout_6 (Dropout)         (None, 121, 121, 64)      0         
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 60, 60, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 58, 58, 32)       

In [289]:
logdir='logs'

In [290]:
train

<_MapDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 18), dtype=tf.float32, name=None))>

In [291]:
tensorboard_callback=tf.keras.callbacks.TensorBoard(log_dir=logdir)
hist=model.fit(train, epochs=500, validation_data=val, callbacks =[tensorboard_callback,early_stopping])

Epoch 1/500


2023-08-02 16:55:15.623336: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_7/dropout_6/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


80/80 [==============================] - 36s 407ms/step - loss: 7.5133 - accuracy: 0.5934 - val_loss: 6.4106 - val_accuracy: 0.5562
Epoch 2/500
80/80 [==============================] - 24s 296ms/step - loss: 4.7973 - accuracy: 0.7730 - val_loss: 4.6937 - val_accuracy: 0.7375
Epoch 3/500
80/80 [==============================] - 24s 288ms/step - loss: 3.1325 - accuracy: 0.8590 - val_loss: 2.9444 - val_accuracy: 0.8562
Epoch 4/500
80/80 [==============================] - 24s 299ms/step - loss: 2.2942 - accuracy: 0.8957 - val_loss: 2.4696 - val_accuracy: 0.8656
Epoch 5/500
80/80 [==============================] - 34s 416ms/step - loss: 2.1026 - accuracy: 0.9023 - val_loss: 1.9923 - val_accuracy: 0.9031
Epoch 6/500
80/80 [==============================] - 24s 294ms/step - loss: 1.7012 - accuracy: 0.9273 - val_loss: 1.7603 - val_accuracy: 0.8875
Epoch 6: early stopping


In [292]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

# Assuming you have already trained the model and have the test dataset
# (or you can use 'test' set if you have already created it)

# Get the true labels and predicted labels from the test set
y_true = []
y_pred = []
for images, labels in test:
    y_true.extend(np.argmax(labels, axis=1))  # Get the true class indices
    y_pred.extend(np.argmax(model.predict(images), axis=1))  # Get the predicted class indices

# Compute the confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)

print("Confusion Matrix:")
print(conf_matrix)

# If you want a classification report (precision, recall, F1-score, etc.), you can use the following:
class_names = os.listdir(r"/kaggle/input/flag-pictures-dataset/Flags")
report = classification_report(y_true, y_pred, target_names=class_names)

print("Classification Report:")
print(report)

1/1 [==============================] - 0s 30ms/step
Confusion Matrix:
[[10  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 12  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 14  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 22  0  0  0  0  0  1  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 18  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 26  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 10  0  0  1  0  0  0  1  0  0  0  0]
 [ 0  0  0  0  0  2  0 19  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  2  0  1 27  1  0  0  0  0  0  0  0  2]
 [ 0  0  0  0  0  1  0  0  0 20  0  1  0  0  0  0  0  0]
 [ 0  0  0  0  0  4  0  0  0  0  9  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  3  0 13  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  3 11  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0 22  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0 22  0  0  0]
 [ 0  0  0  0  1  0  0  0  0  0  0  0  0  0  1  8  0  0]
 [ 0  0  0  0  0  

In [384]:
new_image

In [381]:
new_image_path = r"/kaggle/input/flag-pictures-dataset/Flags/Belgium/belgium-flag-in-grunge-brush-stroke-vector-30915149.jpg"  # Replace with the path to your new image

i=23
# Load and preprocess the new image
new_image = cv2.imread(new_image_path)
new_image = cv2.convertScaleAbs(new_image)
new_image = cv2.cvtColor(new_image, cv2.COLOR_RGB2BGR)
new_image = tf.image.convert_image_dtype(new_image, dtype=tf.uint8)  
resize= tf.image.resize(new_image,(256,256))
# new_image = new_image / 255.0  # Normalize pixel values to [0, 1]
predictions=model.predict(np.expand_dims(resize,axis=0))
# Convert the image to a supported depth (e.g., 8-bit unsigned integers)

#  # Add batch dimension

# View the new image
plt.imshow(new_image.numpy().squeeze())
plt.axis('off')
plt.show()

predicted_class_index = np.argmax(predictions[0])
predicted_class_name = predicted_class_name = sorted(os.listdir(r"/kaggle/input/flag-pictures-dataset/Flags"))[predicted_class_index] # Assuming class indices are integers starting from 0

# Print the predicted class index and name
print("Predicted Class Index:", predicted_class_index)
print("Predicted Class Name:", predicted_class_name)
print("Predicted Class Name:", predictions[0][np.argmax(predictions[0])])
predictions


[ WARN:0@8238.185] global loadsave.cpp:248 findDecoder imread_('https://www.jetpunk.com/img/user-photo-library/15/158a30a34e-450.png'): can't open/read file: check file path/integrity


error: OpenCV(4.8.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [295]:
a=os.listdir(r"/kaggle/input/flag-pictures-dataset/Flags")
sorted(a)

['Argentina',
 'Belgium',
 'Brazil',
 'Canada',
 'Croatia',
 'France',
 'Germany',
 'Latvia',
 'Liberia',
 'Luxemborg',
 'Malaysia',
 'Netherlands',
 'Paraguay',
 'Poland',
 'Serbia',
 'Slovakia',
 'Thailand',
 'USA']

In [296]:
predictions

array([[2.96494123e-02, 5.20711625e-03, 1.11603245e-01, 1.20886811e-03,
        2.00817004e-01, 1.39412656e-01, 1.01676829e-04, 3.17499600e-03,
        8.84200446e-03, 2.32489710e-03, 5.09043515e-04, 1.34732112e-01,
        1.24749787e-01, 3.16268802e-02, 3.88010615e-03, 7.07147270e-02,
        1.30066112e-01, 1.37938303e-03]], dtype=float32)

In [297]:


batch[2][31]

IndexError: tuple index out of range

In [338]:
new_image

<tf.Tensor: shape=(832, 1300, 3), dtype=uint8, numpy=
array([[[ 56,   0,   0],
        [ 56,   0,   0],
        [ 56,   0,   0],
        ...,
        [129,   1,   2],
        [129,   1,   2],
        [129,   1,   2]],

       [[ 56,   0,   0],
        [ 56,   0,   0],
        [ 56,   0,   0],
        ...,
        [129,   1,   2],
        [129,   1,   2],
        [129,   1,   2]],

       [[ 56,   0,   0],
        [ 56,   0,   0],
        [ 56,   0,   0],
        ...,
        [129,   1,   2],
        [129,   1,   2],
        [129,   1,   2]],

       ...,

       [[  1,   1,   1],
        [  1,   1,   1],
        [  1,   1,   1],
        ...,
        [  1,   1,   1],
        [  1,   1,   1],
        [  1,   1,   1]],

       [[  1,   1,   1],
        [  1,   1,   1],
        [  1,   1,   1],
        ...,
        [  1,   1,   1],
        [  1,   1,   1],
        [  1,   1,   1]],

       [[  1,   1,   1],
        [  1,   1,   1],
        [  1,   1,   1],
        ...,
        [  1,   1,  

In [382]:


# Convert the image to a supported depth (e.g., 8-bit unsigned integers)
image = cv2.convertScaleAbs((new_image/(255)).numpy(), alpha=(255.0))
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
image = np.expand_dims(image, axis=0)  # Add batch dimension

# View the new image
plt.imshow(cv2.cvtColor(image.squeeze(), cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()


TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'

In [385]:
model.save("sequential_7")